<a href="https://colab.research.google.com/github/l19060741/Curso-Phyton/blob/main/Anahi_Gonzalez_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes

Notebook elaborado por:

Dra. Jessica Beltrán Márquez

Centro de Investigación en Matemáticas Aplicadas

Universidad Autónoma de Coahuila

Noviembre 2022

In [ ]:
#Importamos las bibliotecas que vamos a utilizar
import math
import random
import pandas as pd
import numpy as np

In [ ]:
# Nombre del archivo csv con el conjunto de datos
dataset_filename = 'irisDataset.csv'    
# Cargar los datos a un dataframe pandas
dataset = pd.read_csv(dataset_filename)
dataset.head()

In [ ]:
#Separar en datos de entrenamiento y de prueba
train_set = dataset.sample(frac = 0.7 , random_state = 0) #Tomar una muestra del conjunto completo
print("Train shape is ", train_set.shape)
test_set = dataset.drop(train_set.index)     #Tomar el complemento de la muestra 
print("Test shape is  ", test_set.shape)


In [ ]:
#Obtener las clases de flores a partir de la última columna
classes = dataset.iloc[:,-1].unique()
print(classes)

In [ ]:
#Esta función obtiene la media y la desviación estándar del dataframe que recibe. 
# Lo hace sobre el primer eje
# Nos sirve para conocer los parámetros de las distribuciones Gaussianas de cada clase
def GaussianParametersFeatures(data):
    means = data.mean(axis = 0) 
    stds = data.std(axis = 0) 
    return means,stds

In [ ]:
# Aplicamos la función sobre nuestro conjunto de datos y obtenemos los promedios y desviaciones estándar 
(means,stds) = GaussianParametersFeatures(dataset[["sepal_length", "sepal_width","petal_length", "petal_width"]])  #No seleccionamos la última columna
print(means)

In [ ]:
dataset.iloc[:,:-1]  #Esta es otra forma de no seleccionar la última columna

# Entrenamiento

Como se vio, con Naive Bayes se entrena un módelo por cada clase

Este modelo se entrena conociendo las probabilidades a priori y las distribuciones de clase


Veamos un ejemplo considerando la clase setosa

In [ ]:
#Primero extraemos solo la información que corresponde a las flores de tipo setosa
train_set_setosa = train_set.loc[train_set.iloc[:,-1] == classes[0]]
print(train_set_setosa)

In [ ]:
#Obtenemos las medias y desviaciones estándar de cada característica, lo que nos sirve para describir la distribución Gaussiana
(means,stds) = GaussianParametersFeatures(train_set_setosa[["sepal_length", "sepal_width","petal_length", "petal_width"]])
print(means)
print(stds)

In [ ]:
#Entrenamiento
#Ahora lo hacemos para las 3 clases distintas: ['setosa' 'versicolor' 'virginica']
#Almacenamos el modelo en un diccionario llamado summaries
summaries = dict()
for className in classes:
    #summaries['classNames'] = classes[i]
    data_subset_class = train_set.loc[train_set.iloc[:,-1] == className]
    prior = len(data_subset_class)/len(train_set)              #<- Probabilidad priori
    (means,stds) = GaussianParametersFeatures(data_subset_class.iloc[:,:-1])
    summaries[className] = (prior,means,stds)

In [ ]:
print(summaries)

In [ ]:
def get_likelihood_normal_distribution(X,mu=0,sigma=1):
  #Calculo de p(x_w) en cada característica
  p_x_w=np.divide(1,(np.sqrt(2*math.pi)*sigma)) * np.exp(-1/2* np.divide(np.power(X-mu, 2),np.multiply(sigma, sigma)))
  return(p_x_w)

In [ ]:
#Este código es solo para probar la función, en este caso con solo una característica
p_x_w = get_likelihood_normal_distribution(4,mu=means[1],sigma=stds[1])
print(p_x_w)

In [ ]:
#Este código es solo para probar la función, en este caso con varias características
p_x_w = get_likelihood_normal_distribution([5.5, 3, 5, 1],mu=means,sigma=stds)
print(p_x_w)

In [ ]:
#Debido a que estamos suponiendo independencia
np.prod(p_x_w)

In [ ]:
#Aplicamos bayes
probs = dict()
def evaluateSample(sample,summaries,classes):
    for className in classes:
        (prior,means,stds) = summaries[className]
        p_x_w = get_likelihood_normal_distribution(sample,mu=means,sigma=stds)
        probs[className] = np.prod(p_x_w)*prior
    return max(probs, key=probs.get)
    

In [ ]:
#Evaluamos sobre un vector
evaluateSample([5, 3, 2, 0.3],summaries,classes)




In [ ]:
#Evaluar un solo ejemplo del testdata
sample=np.array(test_set.iloc[20][0:-1].values.tolist())
evaluateSample(sample,summaries,classes)


In [ ]:
#Evaluar todos los ejemplos del testdata
for row in range(test_set.shape[0]):
    sample=np.array(test_set.iloc[row][0:-1].values.tolist())
    class_sample = test_set.iloc[row][-1]  
    class_predicted = evaluateSample(sample,summaries,classes)
    print("Real: ", class_sample, ", Predicted: ", class_predicted)